## An Automated Portfolio Trading System with Feature Preprocessing and Recurrent Reinforcement Learning

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.data import Data, get_buy_and_hold_returns, INDICATORS

from lib.model import RRL, train, validation

from lib.backtest import (
    run_rrl_strategy, 
    create_backtest_dataset, 
    generate_layout, 
    generate_nrows, 
    generate_subplot_titles, 
    init_subplots, 
    populate_subplots,
    plot_cumulative_profits, 
    generate_df_barplot, 
    make_cumrets_barplot,
)

from lib.metrics import (
    calc_cumulative_profits, 
    calc_sharpe_ratio, 
)

In [3]:
import numpy as np 
import pandas as pd

from pickle import load, dump

### Data

#### Settings

In [136]:
# assets in the portfolio
assets = ["XOM", "VZ", "NKE", "AMAT", "MCD", "MSFT", "AAP", "NOV"]

# date settings
start_date = "2009-12-31"
end_date = "2017-12-29"
window_size = 100

# number of principal components (None means no PCA)
pca_ncp = None

# technical indicators
momentum = [
    # "MOM",
    "MACD",
    # "MFI",
    "RSI"
]

volatility = [
    "ATR",
    # "NATR"
]

cycle = [ 
    # "HTDCP",
    # "HTS",
    # "HTTMMM"
]

volume = [
    "CO",
    "OBV"
]

indicators = momentum + volatility + cycle + volume
indicators

['MACD', 'RSI', 'ATR', 'CO', 'OBV']

#### Instantiate `Data` object

In [137]:
data = Data(start_date, end_date, window_size, assets, indicators, pca_ncp)
data

[*********************100%***********************]  8 of 8 completed


Data(start_date='2009-12-31', end_date='2017-12-29', assets=['XOM', 'VZ', 'NKE', 'AMAT', 'MCD', 'MSFT', 'AAP', 'NOV'], indicators=['MACD', 'RSI', 'ATR', 'CO', 'OBV'], window_size=100)

In [138]:
buy_and_hold_returns = get_buy_and_hold_returns(data.df)

### `RRL` - Training and validation

In [139]:
initial_invest = 100
n_epochs = 50

#### No transaction fees 

$\delta=0$

In [143]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features) 
print(rrl) 

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_no_fees_5ncp.pkl"
file_path = "./backup/no_pca/rrl_no_fees_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=5, delta=0, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  12%|█▏        | 6/50 [00:00<00:01, 23.47it/s, sharpe_ratio=0.299]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.528831748381874
Cumulative profits: 123.7931998836989
Training window: 2010-06-30/2010-10-14


Training in progress...:  14%|█▍        | 7/50 [00:00<00:02, 18.77it/s, sharpe_ratio=2.51]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.3514005111904617
Cumulative profits: 134.34709838495607
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.2580453181255435
Cumulative profits: 215.174229115151
Training window: 2017-03-31/2017-07-18


Training in progress...:  18%|█▊        | 9/50 [00:00<00:01, 23.92it/s, sharpe_ratio=-1.29]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 1.18887199727521
Cumulative profits: 232.85202216180187
Training window: 2017-08-23/2017-12-07


Training in progress...: 100%|██████████| 50/50 [00:01<00:00, 26.08it/s, sharpe_ratio=0.828]


Less than 30 observations in the validation set after preprocessing.


#### Transactions fees 

$\delta \in \{0.001, 0.003, 0.005\}$

In [144]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.001) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_10bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_10bps_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=5, delta=0.001, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  36%|███▌      | 18/50 [00:00<00:00, 33.52it/s, sharpe_ratio=-.0314] 


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.356613700395747
Cumulative profits: 122.40793808736672
Training window: 2010-06-30/2010-10-14


Training in progress...:  36%|███▌      | 18/50 [00:00<00:01, 31.80it/s, sharpe_ratio=2.24]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.4306911148560792
Cumulative profits: 133.19311303605537
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.1839845403825813
Cumulative profits: 203.08541557343366
Training window: 2017-03-31/2017-07-18


Training in progress...:  20%|██        | 10/50 [00:00<00:01, 28.60it/s, sharpe_ratio=-.872]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 1.2627543422478762
Cumulative profits: 217.8862826907766
Training window: 2017-08-23/2017-12-07


Training in progress...:  70%|███████   | 35/50 [00:01<00:00, 27.71it/s, sharpe_ratio=0.604]


Less than 30 observations in the validation set after preprocessing.


In [145]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.003) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_30bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_30bps_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=5, delta=0.003, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  24%|██▍       | 12/50 [00:00<00:01, 25.05it/s, sharpe_ratio=-.209]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.8443826061031094
Cumulative profits: 116.43837457250024
Training window: 2010-06-30/2010-10-14


Training in progress...:  48%|████▊     | 24/50 [00:00<00:00, 31.58it/s, sharpe_ratio=1.81]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 0.7779833423861854
Cumulative profits: 122.38915117263731
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.8039688583452216
Cumulative profits: 104.52879771116825
Training window: 2017-03-31/2017-07-18


Training in progress...:   8%|▊         | 4/50 [00:00<00:01, 25.57it/s, sharpe_ratio=-1.52]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 0.9445658631914593
Cumulative profits: 111.37475327784371
Training window: 2017-08-23/2017-12-07


Training in progress...:   8%|▊         | 4/50 [00:00<00:01, 23.08it/s, sharpe_ratio=1.93] 


Less than 30 observations in the validation set after preprocessing.


In [146]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.005) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_30bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_50bps_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=5, delta=0.005, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  40%|████      | 20/50 [00:00<00:00, 32.68it/s, sharpe_ratio=-.337]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.2668689225586984
Cumulative profits: 110.9406786327149
Training window: 2010-06-30/2010-10-14


Training in progress...:  34%|███▍      | 17/50 [00:00<00:00, 33.66it/s, sharpe_ratio=1.26]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: -0.4268796056144244
Cumulative profits: 107.41640032987762
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.3681929683659328
Cumulative profits: 58.67801010943051
Training window: 2017-03-31/2017-07-18


Training in progress...:  26%|██▌       | 13/50 [00:00<00:01, 27.03it/s, sharpe_ratio=-1.05]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 0.7411921827157404
Cumulative profits: 61.18094165151845
Training window: 2017-08-23/2017-12-07


Training in progress...:  66%|██████▌   | 33/50 [00:01<00:00, 29.89it/s, sharpe_ratio=0.272]


Less than 30 observations in the validation set after preprocessing.


### Backtest

In [147]:
file_names = [f"rrl_{fees}_1" for fees in ("no_fees", "10bps", "30bps", "50bps")]

path = "./backup/no_pca/"
results = {
    f.replace("results_", ""): load(open(f"{path}{f}.pkl", "rb"))
    for f in file_names
}

In [150]:
dfs_backtest = {
    k: create_backtest_dataset(v)
    for k, v in results.items()
}

#### Cumulative profits

In [151]:
initial_invest = 100

In [152]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold_returns, 
    initial_invest, 
    layout)

fig.show()

#### Analysis per trading window for one setting

In [154]:
setting = "rrl_no_fees_1"

df = dfs_backtest[setting]
batch_names = np.unique(df["Batch"])

In [155]:
metric ="Cumulative returns"
fig_title = f"{metric} per trading window | {setting}"

ncols = 4
nrows = generate_nrows(len(batch_names), ncols)

titles = generate_subplot_titles(batch_names)

fig = init_subplots(nrows, ncols, titles)
fig = populate_subplots(fig, df, batch_names, titles, ncols, y_varname=metric, initial_invest=initial_invest)

layout = generate_layout(
    fig_height=1200, 
    fig_width=1400, 
    fig_title=fig_title)
    
fig.update_layout(layout)

In [156]:
df_barplot = generate_df_barplot(df)
title = f"% Cumulative returns at the end of each trading window | {setting}"

layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title=title,
    x_axis_args={"title": {"text": "Trading window"}}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "%"}}
)
fig = make_cumrets_barplot(df_barplot, layout)

fig.show()

#### Portfolio allocation

In [195]:
setting = "rrl_no_fees_1"

positions = results[setting]["positions"]

avg_positions_per_window = np.array([p.mean(axis=1) for p in positions]) 
avg_positions = avg_positions_per_window.mean(axis=0)

df_pie = pd.DataFrame({
    "Asset": results[setting]["assets"], 
    "% Share": 100 * avg_positions
})

In [196]:
import plotly.express as px

palette = px.colors.sequential.Blues
palette.reverse

fig = px.pie(
    df_pie, 
    values="% Share", 
    names="Asset", 
    hole=.4, 
    color_discrete_sequence=palette)

layout = generate_layout(
    fig_height=500, 
    fig_width=700, 
    fig_title="Average portfolio allocation over trading windows"
)
fig.update_layout(layout)

fig.show()